1. 라이브러리 임포트

In [25]:
# Cell 1: Import Libraries
import cv2
import mediapipe as mp
import numpy as np
import time
import tensorflow as tf
from collections import deque
from PIL import Image
import sys # 오류 처리 시 필요

import matplotlib.pyplot as plt
%matplotlib inline

print("Libraries imported successfully.")

Libraries imported successfully.


2. 전역 설정 및 모델, MediaPipe, 웹캠 초기화

In [62]:
flower_gif = Image.open('./img/sunflower.gif')

# 모든 프레임 추출
flower_frames = []
try:
    while True:
        frame = flower_gif.convert('RGBA')
        frame_np = np.array(frame)
        frame_cv = cv2.cvtColor(frame_np, cv2.COLOR_RGBA2BGRA)
        flower_frames.append(np.array(frame_cv))
        flower_gif.seek(flower_gif.tell() + 1)
except EOFError:
    pass

num_flower_frames = len(flower_frames)
frame_idx = 0

# GIF 합성 함수 정의
def get_flower(image, result, frame_idx):
    wrist1 = result.multi_hand_landmarks[0].landmark[0]
    wrist2 = result.multi_hand_landmarks[1].landmark[0]

    wrist1_x = int(wrist1.x * image.shape[1])
    wrist2_x = int(wrist2.x * image.shape[1])
    wrist1_y = int(wrist1.y * image.shape[0])

    hand1 = result.multi_hand_landmarks[0].landmark[9]
    hand2 = result.multi_hand_landmarks[1].landmark[9]

    hand1_x = int(hand1.x * image.shape[1])
    hand2_x = int(hand2.x * image.shape[1])
    hand1_y = int(hand1.y * image.shape[0])
    hand2_y = int(hand2.y * image.shape[0])

    w1 = max(hand1_x, hand2_x) - min(hand1_x, hand2_x)
    w2 = max(hand1_y, hand2_y) - min(hand1_y, hand2_y)

    w = max(w1, w2)
    x = min(wrist1_x, wrist2_x)
    y = wrist1_y

    if w == 0:
        return image

    overlay_img = flower_frames[frame_idx % num_flower_frames]
    overlay_img = cv2.resize(overlay_img, (w, int(w * overlay_img.shape[0] / overlay_img.shape[1])))

    h = overlay_img.shape[0]

    # 이미지 경계 넘어가지 않도록 crop 처리
    y1 = max(0, y - (h//2))
    y2 = min(y1 + h, image.shape[0])
    x1 = max(0, x - (w//2))
    x2 = min(image.shape[1], x1 + w)

    overlay_crop = overlay_img[0:(y2 - y1), 0:(x2 - x1), :] # overlay_img[(h - (y2 - y1)):, (x1 - x):(x2 - x), :]
    mask_crop = overlay_crop[:, :, 3] / 255

    for c in range(3):
        image[y1:y2, x1:x2, c] = \
            (overlay_crop[:, :, c] * mask_crop) + (image[y1:y2, x1:x2, c] * (1 - mask_crop))

    return image

In [ ]:
crown = cv2.imread('./img/crown.png', cv2.IMREAD_UNCHANGED)
crown_ratio = crown.shape[0] / crown.shape[1]

def get_crown(image, result):   # 프레임 이미지, 적용 좌표, 높이 너비, 효과 이미지
    # if result.multi_hand_landmarks is None or len(result.multi_hand_landmarks) < 2:
    #     return image
    hand1 = result.multi_hand_landmarks[0].landmark[0]
    hand2 = result.multi_hand_landmarks[1].landmark[0]

    hand1_x = int(hand1.x * image.shape[1])
    hand2_x = int(hand2.x * image.shape[1])
    hand1_y = int(hand1.y * image.shape[0])

    w = max(hand1_x, hand2_x) - min(hand1_x, hand2_x)
    x = min(hand1_x, hand2_x)
    y = hand1_y

    if w == 0:
        return image # 두 손이 같은 위치면 crown 그리지 않음
    
    overlay_img = crown.copy()
    overlay_img = cv2.resize(overlay_img, (w, int(w*crown_ratio)))

    # alpha = overlay_img[:, :, 3]
    # mask = alpha / 255

    h = overlay_img.shape[0]

    # 이미지 경계 넘는 부분 잘라내기
    y1 = max(0, y - h)
    y2 = y
    x1_crop = max(0, x)
    x2_crop = min(image.shape[1], x + w)

    overlay_crop = overlay_img[(h - (y2 - y1)):, (x1_crop - x):(x2_crop - x), :]
    mask_crop = overlay_crop[:, :, 3] / 255

    for c in range(3):
        image[y1:y2, x1_crop:x2_crop, c] = \
            (overlay_crop[:, :, c] * mask_crop) + (image[y1:y2, x1_crop:x2_crop, c] * (1 - mask_crop))

    return image

In [64]:
# Cell 2: Global Setup and Initialization

# --- 2.1 학습된 모델 로드 ---
model_path = './models/model_v1.h5' # 실제 모델 파일 경로

try:
    model = tf.keras.models.load_model(model_path)
    print(f"Model loaded successfully from '{model_path}'")
except Exception as e:
    print(f"Error loading model from '{model_path}': {e}")
    print("모델 파일이 없거나 경로가 잘못되었습니다. 경로를 확인해주세요.")
    model = None # 모델 로드 실패 시 None으로 설정하여 이후 예측 단계에서 에러 방지


# --- 2.2 MediaPipe Hands 모델 초기화 (두 손 감지 설정) ---
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2, # 두 손 감지 설정
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
print("MediaPipe Hands model initialized for two hands.")


# --- 2.3 웹캠 초기화 ---
cap = cv2.VideoCapture(0) # 기본 웹캠 (대부분 0번)

if not cap.isOpened():
    print("\n--- 오류: 웹캠을 열 수 없습니다! ---")
    print("   웹캠이 연결되어 있는지, 드라이버가 올바른지, 다른 프로그램에서 사용 중이 아닌지 확인해주세요.")
    print("   이 오류가 발생하면 이후 실시간 추론 셀은 작동하지 않습니다.")

else:
    print("웹캠이 성공적으로 초기화되었습니다.")
    # 웹캠 해상도 설정
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    print(f"웹캠 해상도: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}x{int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")


# --- 2.4 제스처 이름 및 시퀀스 길이 설정 ---
# 학습 시 사용했던 actions 리스트와 seq_length 값이 정확히 일치해야 합니다!
actions = [
    'flower', 'crown', 'heart_beat',
    'firework', 'bear', 'cat',
    'son_celebration', 'heart_ont_the_cheek', 'gun',
    'pipe', 'tiger', 'landmarks'
]
seq_length = 30 # 학습 시 사용한 시퀀스 길이와 동일해야 합니다.

print(f"Defined actions for prediction: {actions}")
print(f"Sequence length for model input: {seq_length}")

Model loaded successfully from './models/model_v1.h5'
MediaPipe Hands model initialized for two hands.
웹캠이 성공적으로 초기화되었습니다.
웹캠 해상도: 640x480
Defined actions for prediction: ['flower', 'crown', 'heart_beat', 'firework', 'bear', 'cat', 'son_celebration', 'heart_ont_the_cheek', 'gun', 'pipe', 'tiger', 'landmarks']
Sequence length for model input: 30


3. 실시간 제스처 추론 메인 루프

In [65]:
# Cell 3: Real-time Gesture Prediction Main Loop

# --- 3.1 시퀀스 데이터 저장을 위한 버퍼 초기화 ---
# LSTM 모델은 과거 'seq_length'만큼의 프레임 데이터를 필요로 합니다.
# collections.deque는 고정된 최대 길이를 가지는 큐입니다.
seq_data_buffer = deque(maxlen=seq_length)

# --- 3.2 화면 표시 설정 ---
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_thickness = 2
text_color_default = (255, 255, 255) # White

print("\n--- 실시간 제스처 추론 시작 ---")
print("   웹캠 창이 열리면 화면을 보면서 제스처를 수행하세요.")
print("   'q' 키를 누르면 언제든지 종료할 수 있습니다.")

try:
    if not cap.isOpened() or model is None:
        print("웹캠 또는 모델이 준비되지 않아 추론을 시작할 수 없습니다. 이전 셀의 오류를 확인하세요.")
    else:
        print("카메라 스트림 시작...")
        # 무한 루프를 사용하여 웹캠 프레임을 계속 읽고 처리합니다.
        while True:
            ret, img = cap.read()

            # 프레임 읽기 실패 시 루프 종료
            if not ret:
                print("프레임을 읽을 수 없습니다. 카메라 연결 또는 웹캠 상태를 확인하고 루프를 종료합니다.")
                break

            img = cv2.flip(img, 1) # 좌우 반전 (거울 모드)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # MediaPipe 처리를 위해 BGR -> RGB 변환
            result = hands.process(img_rgb) # MediaPipe를 이용한 손 랜드마크 감지
            img = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR) # OpenCV 표시를 위해 RGB -> BGR 변환

            # --- 3.3 현재 프레임의 특징 데이터 추출 및 패딩 ---
            # 학습 시 사용했던 두 손 데이터를 구성하는 로직과 동일해야 합니다.
            current_frame_features = []
            num_detected_hands = 0
            display_message = ""
            current_text_color = text_color_default

            if result.multi_hand_landmarks:
                num_detected_hands = len(result.multi_hand_landmarks)
                # 두 손이 모두 감지되었을 때만 실제 특징 데이터 처리
                if num_detected_hands == 2:
                    # 손의 x좌표를 기준으로 정렬하여 항상 일관된 순서(예: 왼손, 오른손)로 데이터를 구성
                    handedness_sorted = [(lm.landmark[0].x, i) for i, lm in enumerate(result.multi_hand_landmarks)]
                    handedness_sorted.sort()

                    for _, hand_idx in handedness_sorted:
                        res = result.multi_hand_landmarks[hand_idx]

                        # 랜드마크 추출 (21개 관절 * 4차원 = 84개 특징)
                        joint = np.zeros((21, 4))
                        for j, lm in enumerate(res.landmark):
                            joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                        current_frame_features.extend(joint.flatten().tolist())

                        # 관절 각도 계산 (15개 특징)
                        v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3]
                        v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3]
                        v = v2 - v1
                        norm_v = np.linalg.norm(v, axis=1)
                        v = v / (norm_v[:, np.newaxis] + 1e-8) # 0으로 나누는 오류 방지
                        dot_product = np.einsum('nt,nt->n',
                                                 v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:],
                                                 v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])
                        dot_product = np.clip(dot_product, -1.0, 1.0) # arccos 도메인 유지
                        angle = np.degrees(np.arccos(dot_product))
                        current_frame_features.extend(angle.tolist())

                        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS) # 화면에 랜드마크 그리기

                    # 두 손의 특징 데이터가 모두 추출되었는지 최종 확인 (총 198개)
                    expected_features_len = (21 * 4 + 15) * 2
                    if len(current_frame_features) != expected_features_len:
                        print(f"특징 개수 불일치! (예상: {expected_features_len}, 실제: {len(current_frame_features)})")
                        # 불일치 시 0으로 패딩하여 모델 입력 shape 유지
                        current_frame_features = np.zeros(expected_features_len, dtype=np.float32).tolist()
                        display_message = "Data Error - Padding"
                        current_text_color = (0, 0, 255) # Red
                else:
                    # 손이 감지되었지만 2개가 아닌 경우 (1개 또는 3개 이상)
                    expected_features_len = (21 * 4 + 15) * 2 # 항상 198개
                    current_frame_features = np.zeros(expected_features_len, dtype=np.float32).tolist()
                    display_message = f"Detected {num_detected_hands} hand(s). Need 2."
                    current_text_color = (0, 165, 255) # Orange
            else:
                # 손이 전혀 감지되지 않은 경우
                expected_features_len = (21 * 4 + 15) * 2 # 항상 198개
                current_frame_features = np.zeros(expected_features_len, dtype=np.float32).tolist()
                display_message = "No hands detected."
                current_text_color = (0, 0, 255) # Red

            # 추출된 (또는 패딩된) 특징 데이터를 시퀀스 버퍼에 추가
            seq_data_buffer.append(np.array(current_frame_features, dtype=np.float32))


            # --- 3.4 시퀀스 버퍼가 충분히 채워지면 예측 수행 ---
            if len(seq_data_buffer) == seq_length:
                # 모델 입력 형태에 맞게 차원 확장 (배치 차원 추가)
                input_sequence = np.expand_dims(np.array(seq_data_buffer), axis=0) # (1, seq_length, num_features)

                if model is not None: # 모델이 성공적으로 로드되었는지 확인
                    preds = model.predict(input_sequence, verbose=0)[0] # verbose=0으로 예측 진행바 숨기기
                    action_idx = np.argmax(preds)
                    confidence = preds[action_idx]

                    # 신뢰도가 높은 예측만 표시 ( threshold 조절 가능)
                    if confidence > 0.7: # 70% 이상의 신뢰도일 때만 제스처 이름 표시
                        predicted_action = actions[action_idx]
                        display_message = f"Action: {predicted_action} ({confidence:.2f})"
                        current_text_color = (0, 255, 0) # Green
                        if predicted_action == 'crown':
                            img = get_crown(img, result)
                        elif predicted_action == 'flower':
                            img = get_flower(img, result, frame_idx)
                            frame_idx += 1
                    else:
                        display_message = "Action: Thinking..." # 신뢰도가 낮으면 "생각 중"으로 표시
                        current_text_color = (0, 165, 255) # Orange
                else:
                    display_message = "Model not loaded." # 모델이 로드되지 않았을 경우
                    current_text_color = (0, 0, 255) # Red
            else:
                # 버퍼가 채워지는 동안 메시지 표시
                display_message = f"Buffering... ({len(seq_data_buffer)}/{seq_length})"
                current_text_color = (255, 255, 0) # Yellow


            # --- 3.5 웹캠 화면에 정보 표시 및 업데이트 ---
            cv2.putText(img, display_message, (10, 60), font, font_scale, current_text_color, font_thickness)
            cv2.imshow('Gesture Recognition (Two Hands)', img)

            # --- 3.6 종료 조건 확인 ('q' 키) ---
            key = cv2.waitKey(1) & 0xFF # 1ms 대기 및 키 입력 확인
            if key == ord('q'):
                print("사용자 요청 ('q' 키)으로 추론을 중단합니다.")
                break
            # 참고: 창 닫기 버튼으로 종료하는 기능은 Jupyter 환경에서 불안정할 수 있습니다.
            # `if cv2.getWindowProperty('Gesture Recognition (Two Hands)', cv2.WND_PROP_VISIBLE) < 1:` 와 같은 코드는
            # 특정 환경에서 오류를 유발할 수 있어 일반적으로 'q' 키를 권장합니다.

except Exception as e:
    print(f"\n--- 추론 중 치명적인 오류 발생 ---")
    print(f"   오류 내용: {e}")
    import traceback
    traceback.print_exc() # 상세한 에러 스택 트레이스 출력 (디버깅에 유용)

finally:
    # --- 3.7 리소스 정리 (예외 발생 여부와 상관없이 항상 실행) ---
    print("\n--- 리소스 정리 중... ---")
    if 'cap' in locals() and cap.isOpened():
        cap.release()
        print("   웹캠 리소스가 해제되었습니다.")
    else:
        print("   웹캠이 이미 닫혀 있거나 초기화되지 않았습니다.")

    cv2.destroyAllWindows() # 모든 OpenCV 창 닫기
    print("   모든 OpenCV 창이 닫혔습니다.")

    if 'hands' in locals():
        hands.close() # MediaPipe Hands 모델 리소스 해제
        print("   MediaPipe Hands 모델 리소스가 해제되었습니다.")

print("\n추론 스크립트 실행 완료.")


--- 실시간 제스처 추론 시작 ---
   웹캠 창이 열리면 화면을 보면서 제스처를 수행하세요.
   'q' 키를 누르면 언제든지 종료할 수 있습니다.
카메라 스트림 시작...

--- 추론 중 치명적인 오류 발생 ---
   오류 내용: list index out of range

--- 리소스 정리 중... ---


Traceback (most recent call last):
  File "C:\Users\samba\AppData\Local\Temp\ipykernel_32628\322625665.py", line 117, in <module>
    img = get_crown(img, result)
  File "C:\Users\samba\AppData\Local\Temp\ipykernel_32628\3225131281.py", line 6, in get_crown
    hand2 = result.multi_hand_landmarks[1].landmark[0]
IndexError: list index out of range


   웹캠 리소스가 해제되었습니다.
   모든 OpenCV 창이 닫혔습니다.
   MediaPipe Hands 모델 리소스가 해제되었습니다.

추론 스크립트 실행 완료.
